# 한국어 NSMC 감성분류

- https://github.com/e9t/nsmc 는 네이버 영화 리뷰 코퍼스로, Train 데이터셋과 Text 데이터셋으로 구성되어 있다.
- 먼저 해당 GitHub으로부터 데이터를 Clone하라.
- Clone 후 Train 데이터셋은 5,000건 Test 데이터셋은 500건을 활용하라. (훈련시간 이슈; 하드웨어 여유가 있다면 그 이상도 가능 But No Extra Point)
- https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라 한국어 감성분석 모델을 만들어라



## 목표


- 깃헙 내의 txt 파일을 불러온 후 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 영화 리뷰들의 분류 결과를 보이기

- 참고 사이트
    
    - https://pytorch.org/text/
    - http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    - https://github.com/pytorch/text
    - https://mc.ai/using-fine-tuned-gensim-word2vec-embeddings-with-torchtext-and-pytorch/
    - https://github.com/bentrevett

## 제출


- 이메일: 장동준 qwer4107@snu.ac.kr
- 마감: 2024년 5월 27일 월요일 오후 11시 59분 59초까지!

- **주의사항**
    1) .ipynb 파일로 제출할 것
    2) Colab으로 구축했을 시 특히 런타임 초기화 후 코드 전체 실행했을 때 오류 없는 지 확인!
    3) User Input 부분의 결과를 잘 나타낼 것
    4) 새로운 모듈 및 라이브러리 설치 시 # !pip로 해당 모듈 명시

## 정리
- 구현한 시스템의 성능을 정리


In [1]:
## 프로그래밍 시작

In [2]:
pip install portalocker

In [3]:
!git clone https://github.com/e9t/nsmc

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1
Receiving objects: 100% (14763/14763), 56.19 MiB | 25.11 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [4]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 49.5 MB/s eta 0:00:00


In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00


In [6]:
import torch
import pandas as pd
import math
import collections
import tqdm
from torch.utils.data import IterableDataset
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe
from konlpy.tag import Okt
import datasets
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [7]:
torch.__version__

'2.3.0+cu121'

In [8]:
seed = 2024
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [9]:
torch.cuda.get_device_name(device = 0)

'Tesla T4'

#전처리

## 전처리 함수 이전_ 쌩 날 것 살펴보기

In [15]:
train_df = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test_df = pd.read_csv("nsmc/ratings_test.txt", sep='\t')
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [14]:
# 공백만 있는 행 찾기
only_whitespace_rows = train_df[train_df['document'].str.strip() == '']
only_whitespace_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        0 non-null      int64 
 1   document  0 non-null      object
 2   label     0 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 0.0+ bytes


In [17]:
# 중복된 행 찾기
# 엥 테스트에도 중복 있네?
duplicated_rows_train = train_df[train_df.duplicated(subset=['document'], keep=False)]
duplicated_rows_train.info()

duplicated_rows_test = test_df[test_df.duplicated(subset=['document'], keep=False)]
duplicated_rows_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4954 entries, 54 to 149987
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4954 non-null   int64 
 1   document  4949 non-null   object
 2   label     4954 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 154.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1126 entries, 18 to 49934
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1126 non-null   int64 
 1   document  1123 non-null   object
 2   label     1126 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 35.2+ KB


In [18]:
duplicated_rows_test
# 아니 "굿" 이 반복된 걸로 나오는 구나..

,id,document,label
18,6406912,나이스,1
78,2854944,볼만함,1
116,6910486,.,1
160,7677681,굿,1
219,1050028,글쎄,0
...,...,...,...
49731,7437033,별로,0
49760,5890883,최악이다,0
49767,4032953,최고,1
49841,8097768,2년이 지나도 잊혀지지 않는 소중한 드라마.,1


In [20]:
#다섯글자 이상 중복된 문장
duplicated_rows_over_five = duplicated_rows_test[duplicated_rows_test['document'].str.len() >= 5]

print(duplicated_rows_over_five)
print(duplicated_rows_over_five.info())

# "재미가 없다" => 중복 되었구나..

             id                  document  label
288      791996                   재미가 없다.      0
312       23174                     재미있어요      1
422     8483345            tv 전기세가 아깝다!!!      0
716     8013115                   평점조절위원회      0
718     2460341                     따뜻한영화      1
...         ...                       ...    ...
49130  10138463                노잼노잼노잼노잼노잼      0
49540   7009210                     쓰레기영화      0
49563   2192893                     good!      0
49671   7627982                    재미있어요.      1
49841   8097768  2년이 지나도 잊혀지지 않는 소중한 드라마.      1

[327 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 327 entries, 288 to 49841
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        327 non-null    int64 
 1   document  327 non-null    object
 2   label     327 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 10.2+ KB
None


In [21]:
train_df_said_good = train_df['document'] == '굿'
train_df_said_good.value_counts()

document
False    149819
True        181
Name: count, dtype: int64

질문 : txt 데이터 받으면 해야할 일들..?
1. 중복
2. 결측치?

## 전처리 함수 적용

###전처리 함수 Ver1

In [22]:
# 데이터프레임 전처리
def preprocess_dataframe(input_dataframe:pd.DataFrame)->pd.DataFrame:
    return_dataframe = input_dataframe.copy(deep=True)
    return_dataframe.drop_duplicates(subset=['document'], inplace=True)

# 질문 : 전처리는 중복만 제거한 겁니까??



    # 한글, 공백만 포함
    # return_dataframe['korean'] = return_dataframe['document'].str.replace(pat=r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", repl=r"", regex=True)
    # 영어도 포함시킬려면
    # return_dataframe['document'] = return_dataframe['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")
    # 공백만 존재할 경우 na로 대체한다.
    # return_dataframe['korean'] = return_dataframe['korean'].replace('', np.nan)
    # na만 존재하는 행들을 제거한다
    return_dataframe = return_dataframe.dropna(how='any')
    return return_dataframe

In [23]:
train_df = pd.read_table("nsmc/ratings_train.txt")
test_df = pd.read_table("nsmc/ratings_test.txt")
len(train_df), len(test_df)

(150000, 50000)

In [24]:
preprocessed_train_df = preprocess_dataframe(train_df)
preprocessed_test_df = preprocess_dataframe(test_df)
preprocessed_train_df.shape, preprocessed_test_df.shape

((146182, 3), (49157, 3))

In [25]:
preprocessed_train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


### 전처리 함수 Ver 2

In [26]:
# 데이터프레임 전처리
def preprocess_dataframe_ver2(input_dataframe:pd.DataFrame)->pd.DataFrame:
    return_dataframe = input_dataframe.copy(deep=True)
    return_dataframe.drop_duplicates(subset=['document'], inplace=True)

# 질문 : 전처리는 중복만 제거한 겁니까??



    # 한글, 공백만 포함 #아 특수기호가 사라지는 구나..!
    return_dataframe['korean'] = return_dataframe['document'].str.replace(pat=r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", repl=r"", regex=True)
    # 영어도 포함시킬려면
    return_dataframe['document'] = return_dataframe['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")
    # 공백만 존재할 경우 na로 대체한다.
    # return_dataframe['korean'] = return_dataframe['korean'].replace('', np.nan)
    # na만 존재하는 행들을 제거한다
    return_dataframe = return_dataframe.dropna(how='any')
    return return_dataframe

In [27]:
preprocessed_train_df = preprocess_dataframe_ver2(train_df)
preprocessed_train_df
# 한글 + 공백 = 특수기호 사라짐 "korean 열 주목

,id,document,label,korean
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 솔직히 재미는 없다평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
...,...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0,인간이 문제지 소는 뭔죄인가
149996,8549745,평점이 너무 낮아서...,1,평점이 너무 낮아서
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1,청춘 영화의 최고봉방황과 우울했던 날들의 자화상


In [28]:
preprocessed_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146182 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        146182 non-null  int64 
 1   document  146182 non-null  object
 2   label     146182 non-null  int64 
 3   korean    146182 non-null  object
dtypes: int64(2), object(2)
memory usage: 5.6+ MB


In [29]:
preprocessed_test_df = preprocess_dataframe_ver2(test_df)
preprocessed_test_df

,id,document,label,korean
0,6270596,굳 ㅋ,1,굳 ㅋ
1,9274899,GDNTOPCLASSINTHECLUB,0,
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠
...,...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네


### DataFrame을 datasets로 변환

In [30]:
# 각 데이터프레임을 datasets로 표현
train_data = datasets.Dataset.from_pandas(preprocessed_train_df)
test_data = datasets.Dataset.from_pandas(preprocessed_test_df)

In [31]:
test_data

Dataset({
    features: ['id', 'document', 'label', 'korean', '__index_level_0__'],
    num_rows: 49157
})

#토큰화

In [32]:
kor_tokenizer = get_tokenizer(Okt().morphs)

## 토근화 Ver1 : documnet 행에 대해서 토큰화 : 미사용

In [ ]:
# 앞에서 설정한 불용어를 제거하고, otk tokenizer를 사용하여 리뷰 데이터를 tokenize한다
def kor_tokenize(review, tokenizer, max_length):
    tokens = tokenizer(review["document"])[:max_length]
    length = len(tokens)
    return {"tokens": tokens, "length": length}

max_length=30

15분 걸리네..

In [ ]:
train_data = train_data.map(
    kor_tokenize, fn_kwargs={"tokenizer": kor_tokenizer, "max_length": max_length}
)

#Document 행을 토큰화 했다.

Parameter 'fn_kwargs'={'tokenizer': <bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7b9c706b34c0>>, 'max_length': 30} of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/146182 [00:00<?, ? examples/s]

토큰화 기준이 뭐지?
토큰화 패키지의 디폴트 값을 사용한건가?
뭐야~~

In [ ]:
train_data[10]

{'id': 9008700,
 'document': '걍인피니트가짱이다.진짜짱이다♥',
 'label': 1,
 'korean': '걍인피니트가짱이다진짜짱이다',
 '__index_level_0__': 10,
 'tokens': ['걍', '인피니트', '가', '짱', '이다', '.', '진짜', '짱', '이다', '♥'],
 'length': 10}

In [ ]:
preprocessed_test_df.head()

,id,document,label,korean
0,6270596,굳 ㅋ,1,굳 ㅋ
1,9274899,GDNTOPCLASSINTHECLUB,0,
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠


## 토근화 Ver2 'korean' 행에 대해서 토큰화 : 사용

In [33]:
def kor_tokenize_korean(review, tokenizer, max_length):
    tokens = tokenizer(review["korean"])[:max_length]
    length = len(tokens)
    return {"tokens": tokens, "length": length}


In [35]:
max_length=30
train_data_korean = train_data.map(
    kor_tokenize_korean, fn_kwargs={"tokenizer": kor_tokenizer, "max_length": max_length}
)

# 맵할 때 함수를 잘 입력하렴.. #kor_tokenize_korean

Parameter 'fn_kwargs'={'tokenizer': <bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7802f81442b0>>, 'max_length': 30} of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/146182 [00:00<?, ? examples/s]

In [37]:
train_data_korean[10]
# 인터넷에 따르면, 특수문자 단순 제거하지 말라고 했는데... 해버렸다.. 한번 해보자..

{'id': 9008700,
 'document': '걍인피니트가짱이다.진짜짱이다♥',
 'label': 1,
 'korean': '걍인피니트가짱이다진짜짱이다',
 '__index_level_0__': 10,
 'tokens': ['걍', '인피니트', '가', '짱', '이다', '진짜', '짱', '이다'],
 'length': 8}

In [38]:
max_length=30
test_data_korean = test_data.map(
    kor_tokenize_korean, fn_kwargs={"tokenizer": kor_tokenizer, "max_length": max_length}
)

Map:   0%|          | 0/49157 [00:00<?, ? examples/s]

In [60]:
train_data = train_data_korean
test_data = test_data_korean

In [62]:
train_data[10]

{'id': 9008700,
 'document': '걍인피니트가짱이다.진짜짱이다♥',
 'label': 1,
 'korean': '걍인피니트가짱이다진짜짱이다',
 '__index_level_0__': 10,
 'tokens': ['걍', '인피니트', '가', '짱', '이다', '진짜', '짱', '이다'],
 'length': 8}

In [63]:
# 불용어만 존재하여 토큰이 없는 데이터를 삭제
def filter_empty_tokens(example):
    return len(example["tokens"]) > 0

train_data = train_data.filter(filter_empty_tokens)
test_data = test_data.filter(filter_empty_tokens)

Filter:   0%|          | 0/146182 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49157 [00:00<?, ? examples/s]

In [65]:
train_data[10]

{'id': 9008700,
 'document': '걍인피니트가짱이다.진짜짱이다♥',
 'label': 1,
 'korean': '걍인피니트가짱이다진짜짱이다',
 '__index_level_0__': 10,
 'tokens': ['걍', '인피니트', '가', '짱', '이다', '진짜', '짱', '이다'],
 'length': 8}

#Train-Val split, 단어집합 구축 & Mapping

In [66]:
# 학습 데이터를 8:2로 나누어 학습 데이터, 검증 데이터로 분리
test_size = 0.2
train_valid_data = train_data.train_test_split(test_size=test_size)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]

In [67]:
train_data[10]

{'id': 2091153,
 'document': '뭐 이런 쓰레기 같은 영화가 -_-',
 'label': 0,
 'korean': '뭐 이런 쓰레기 같은 영화가 ',
 '__index_level_0__': 118469,
 'tokens': ['뭐', '이런', '쓰레기', '같은', '영화', '가'],
 'length': 6}

In [68]:
# 아 언노운 토큰은 train에서만 써야하는 구만...! not in val set
special_tokens = ["<unk>", "<pad>"]
min_freq = 5

vocab = build_vocab_from_iterator(
    train_data["tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [69]:
unk_index = vocab["<unk>"]
pad_index = vocab["<pad>"]

In [71]:
vocab.set_default_index(unk_index)

In [72]:
#여기서 글자를 숫자로 만드는구나...
def numericalize(example, vocab):
    ids = vocab.lookup_indices(example["tokens"])
    return {"ids": ids}

In [73]:
train_data[10]

{'id': 2091153,
 'document': '뭐 이런 쓰레기 같은 영화가 -_-',
 'label': 0,
 'korean': '뭐 이런 쓰레기 같은 영화가 ',
 '__index_level_0__': 118469,
 'tokens': ['뭐', '이런', '쓰레기', '같은', '영화', '가'],
 'length': 6}

In [74]:
# 리뷰 데이터들을 vocab에 존재하는 단어들로 mapping한다.
# 이제 글자 => 숫자!!
train_data = train_data.map(numericalize, fn_kwargs={"vocab": vocab})
valid_data = valid_data.map(numericalize, fn_kwargs={"vocab": vocab})
test_data = test_data.map(numericalize, fn_kwargs={"vocab": vocab})

Map:   0%|          | 0/116314 [00:00<?, ? examples/s]

Map:   0%|          | 0/29079 [00:00<?, ? examples/s]

Map:   0%|          | 0/48852 [00:00<?, ? examples/s]

In [81]:
target_data = next(data for data in train_data if data["id"] == 2091153)
print(target_data)
print('이렇게 숫자로 변한걸까?: ',target_data["ids"])

{'id': 2091153, 'document': '뭐 이런 쓰레기 같은 영화가 -_-', 'label': 0, 'korean': '뭐 이런 쓰레기 같은 영화가 ', '__index_level_0__': 118469, 'tokens': ['뭐', '이런', '쓰레기', '같은', '영화', '가'], 'length': 6, 'ids': [64, 30, 69, 89, 3, 5]}
이렇게 숫자로 변한걸까?:  [64, 30, 69, 89, 3, 5]


In [82]:
# 데이터셋을 torch 형태로 변환한다.
train_data = train_data.with_format(type="torch", columns=["ids", "label", "length"])
valid_data = valid_data.with_format(type="torch", columns=["ids", "label", "length"])
test_data = test_data.with_format(type="torch", columns=["ids", "label", "length"])

In [87]:
print(train_data[10]["ids"])
print('뭐 이런 쓰레기 같은 영화가 -_-')

tensor([64, 30, 69, 89,  3,  5])
뭐 이런 쓰레기 같은 영화가 -_-


In [88]:
train_data[0]

{'label': tensor(1),
 'length': tensor(7),
 'ids': tensor([   0,  421, 1657,    0, 4131,  798,   91])}

# Mini Batch 만들기 & 최종 데이터 준비

In [89]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_ids = [i["ids"] for i in batch]
        batch_ids = nn.utils.rnn.pad_sequence(
            batch_ids, padding_value=pad_index, batch_first=True
        )
        batch_length = [i["length"] for i in batch]
        batch_length = torch.stack(batch_length)
        batch_label = [i["label"] for i in batch]
        batch_label = torch.stack(batch_label)
        batch = {"ids": batch_ids, "length": batch_length, "label": batch_label}
        return batch

    return collate_fn

In [90]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [91]:
# 각 데이터를 data_loader로 변환하여 모델에 실을 준비를 한다.
batch_size = 512

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
# 이젠 미니배치로 떠나서 볼 수 없게 되었다..
#train_data_loader[10]

In [94]:
vectors = GloVe()
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())

.vector_cache/glove.840B.300d.zip: 2.18GB [06:55, 5.24MB/s]                            
100%|█████████▉| 2196016/2196017 [04:23<00:00, 8322.32it/s]


# NBoW(Neaural Bag of Words) 모델

In [95]:
class NBoW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, ids):
        # ids = [batch size, seq len]
        embedded = self.embedding(ids)
        # embedded = [batch size, seq len, embedding dim]
        pooled = embedded.mean(dim=1)
        # pooled = [batch size, embedding dim]
        prediction = self.fc(pooled)
        # prediction = [batch size, output dim]
        return prediction

In [96]:
vocab_size = len(vocab)
embedding_dim = 300
output_dim = len(train_data.unique("label"))

NBoW_model = NBoW(vocab_size, embedding_dim, output_dim, pad_index)

In [97]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(NBoW_model):,} trainable parameters")

The model has 5,305,502 trainable parameters


In [98]:
NBoW_model.embedding.weight.data = pretrained_embedding

In [100]:
lr = 5e-4
# 로스 : 크로스 앤트로피
# 아담으로 최적화!!!
optimizer = optim.Adam(NBoW_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [101]:
# cuda 사용함
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [102]:
NBoW_model = NBoW_model.to(device)
criterion = criterion.to(device)

In [103]:
def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

In [104]:
def train_NBoW(data_loader, model, criterion, optimizer, device):
    model.train()
    epoch_losses = []
    epoch_accs = []
    for batch in tqdm.tqdm(data_loader, desc="training..."):
        ids = batch["ids"].to(device)
        label = batch["label"].to(device)
        prediction = model(ids)
        loss = criterion(prediction, label)
        accuracy = get_accuracy(prediction, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())
    return np.mean(epoch_losses), np.mean(epoch_accs)

In [105]:
def evaluate_NBoW(data_loader, model, criterion, device):
    model.eval()
    epoch_losses = []
    epoch_accs = []
    with torch.no_grad():
        for batch in tqdm.tqdm(data_loader, desc="evaluating..."):
            ids = batch["ids"].to(device)
            label = batch["label"].to(device)
            prediction = model(ids)
            loss = criterion(prediction, label)
            accuracy = get_accuracy(prediction, label)
            epoch_losses.append(loss.item())
            epoch_accs.append(accuracy.item())
    return np.mean(epoch_losses), np.mean(epoch_accs)

In [106]:
n_epochs = 10
best_valid_loss = float("inf")

metrics = collections.defaultdict(list)

for epoch in range(n_epochs):
    train_loss, train_acc = train_NBoW(
        train_data_loader, NBoW_model, criterion, optimizer, device
    )
    valid_loss, valid_acc = evaluate_NBoW(valid_data_loader, NBoW_model, criterion, device)
    metrics["train_losses"].append(train_loss)
    metrics["train_accs"].append(train_acc)
    metrics["valid_losses"].append(valid_loss)
    metrics["valid_accs"].append(valid_acc)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(NBoW_model.state_dict(), "base_nbow.pt")
    print(f"epoch: {epoch}")
    print(f"train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}")
    print(f"valid_loss: {valid_loss:.3f}, valid_acc: {valid_acc:.3f}")

evaluating...: 100%|██████████| 57/57 [00:01<00:00, 42.65it/s]


epoch: 0
train_loss: 0.610, train_acc: 0.764
valid_loss: 0.504, valid_acc: 0.801


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 40.66it/s]


epoch: 1
train_loss: 0.438, train_acc: 0.825
valid_loss: 0.412, valid_acc: 0.827


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 42.65it/s]


...

epoch: 7
train_loss: 0.287, train_acc: 0.882
valid_loss: 0.371, valid_acc: 0.842


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 42.37it/s]


epoch: 8
train_loss: 0.280, train_acc: 0.885
valid_loss: 0.375, valid_acc: 0.840


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 36.82it/s]

epoch: 9
train_loss: 0.274, train_acc: 0.887
valid_loss: 0.380, valid_acc: 0.840


In [107]:
_, test_acc = evaluate_NBoW(test_data_loader, NBoW_model, criterion, device)
print(f"test_acc: {test_acc:.3f}")
# 특수문자 무지성으로 뻈더니 조금 성능이 감소한걸까?? 0.842 -> 0.839

evaluating...: 100%|██████████| 96/96 [00:02<00:00, 42.79it/s]

test_acc: 0.839


# LSTM

## LSTM 모델 준비

In [108]:
class LSTM(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,
        output_dim,
        n_layers,
        bidirectional,
        dropout_rate,
        pad_index,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            n_layers,
            bidirectional=bidirectional,
            dropout=dropout_rate,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, ids, length):
        # ids = [batch size, seq len]
        # length = [batch size]
        embedded = self.dropout(self.embedding(ids))
        # embedded = [batch size, seq len, embedding dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, length, batch_first=True, enforce_sorted=False
        )
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        output, output_length = nn.utils.rnn.pad_packed_sequence(packed_output)
        # output = [batch size, seq len, hidden dim * n directions]
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))
            # hidden = [batch size, hidden dim * 2]
        else:
            hidden = self.dropout(hidden[-1])
            # hidden = [batch size, hidden dim]
        prediction = self.fc(hidden)
        # prediction = [batch size, output dim]
        return prediction

In [109]:
vocab_size = len(vocab)
embedding_dim = 300
hidden_dim = 128
output_dim = len(train_data.unique("label"))
n_layers = 4
bidirectional = True
dropout_rate = 0.2

lstm_model = LSTM(
    vocab_size,
    embedding_dim,
    hidden_dim,
    output_dim,
    n_layers,
    bidirectional,
    dropout_rate,
    pad_index,
)

In [110]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(lstm_model):,} trainable parameters")

The model has 6,931,526 trainable parameters


In [111]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if "bias" in name:
                nn.init.zeros_(param)
            elif "weight" in name:
                nn.init.orthogonal_(param)

In [112]:
lstm_model.apply(initialize_weights)

LSTM(
  (embedding): Embedding(17683, 300, padding_idx=1)
  (lstm): LSTM(300, 128, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [113]:
lstm_model.embedding.weight.data = pretrained_embedding

In [114]:
lr = 5e-4

optimizer = optim.Adam(lstm_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [115]:
# cuda 사용함
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [116]:
lstm_model = lstm_model.to(device)
criterion = criterion.to(device)

In [117]:
def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

In [118]:
def train(dataloader, model, criterion, optimizer, device):
    model.train()
    epoch_losses = []
    epoch_accs = []
    for batch in tqdm.tqdm(dataloader, desc="training..."):
        ids = batch["ids"].to(device)
        length = batch["length"]
        label = batch["label"].to(device)
        prediction = model(ids, length)
        loss = criterion(prediction, label)
        accuracy = get_accuracy(prediction, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())
    return np.mean(epoch_losses), np.mean(epoch_accs)

In [119]:
def evaluate(dataloader, model, criterion, device):
    model.eval()
    epoch_losses = []
    epoch_accs = []
    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader, desc="evaluating..."):
            ids = batch["ids"].to(device)
            length = batch["length"]
            label = batch["label"].to(device)
            prediction = model(ids, length)
            loss = criterion(prediction, label)
            accuracy = get_accuracy(prediction, label)
            epoch_losses.append(loss.item())
            epoch_accs.append(accuracy.item())
    return np.mean(epoch_losses), np.mean(epoch_accs)

## LSTM 학습 및 평가

In [120]:
n_epochs = 10
best_valid_loss = float("inf")

metrics = collections.defaultdict(list)

for epoch in range(n_epochs):
    train_loss, train_acc = train(
        train_data_loader, lstm_model, criterion, optimizer, device
    )
    valid_loss, valid_acc = evaluate(valid_data_loader, lstm_model, criterion, device)
    metrics["train_losses"].append(train_loss)
    metrics["train_accs"].append(train_acc)
    metrics["valid_losses"].append(valid_loss)
    metrics["valid_accs"].append(valid_acc)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), "base_lstm.pt")
    print(f"epoch: {epoch}")
    print(f"train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}")
    print(f"valid_loss: {valid_loss:.3f}, valid_acc: {valid_acc:.3f}")

evaluating...: 100%|██████████| 57/57 [00:02<00:00, 25.61it/s]


epoch: 0
train_loss: 0.415, train_acc: 0.802
valid_loss: 0.358, valid_acc: 0.842


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 28.87it/s]


epoch: 1
train_loss: 0.313, train_acc: 0.865
valid_loss: 0.362, valid_acc: 0.840


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 30.04it/s]


...

epoch: 7
train_loss: 0.174, train_acc: 0.928
valid_loss: 0.483, valid_acc: 0.824


evaluating...: 100%|██████████| 57/57 [00:02<00:00, 28.19it/s]


epoch: 8
train_loss: 0.155, train_acc: 0.937
valid_loss: 0.549, valid_acc: 0.820


evaluating...: 100%|██████████| 57/57 [00:01<00:00, 29.30it/s]

epoch: 9
train_loss: 0.135, train_acc: 0.946
valid_loss: 0.629, valid_acc: 0.822


In [121]:
_, test_acc = evaluate(test_data_loader, lstm_model, criterion, device)
print(f"test_acc: {test_acc:.3f}")
# 오 이거는 0.820 -> 0.823

evaluating...: 100%|██████████| 96/96 [00:04<00:00, 22.24it/s]

test_acc: 0.823


# User Input : 최종 시험 NBoW vs LSTM


In [122]:
def predict_sentiment(model, sentence, min_len=5):
    sentence = re.sub(pattern=r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", repl=r"", string=sentence)
    tokens = kor_tokenizer(sentence)
    # tokens = [token for token in tokens if token not in stop_words]
    ids = vocab.lookup_indices(tokens)
    tensor = torch.LongTensor(ids).unsqueeze(dim=0).to(device)
    if model.__class__.__name__==NBoW.__name__:
        prediction = model(tensor).squeeze(dim=0)
    if model.__class__.__name__==LSTM.__name__:
        length = torch.LongTensor([len(ids)])
        prediction = model(tensor, length).squeeze(dim=0)
    probability = torch.softmax(prediction, dim=-1)
    predicted_class = prediction.argmax(dim=-1).item()
    predicted_probability = probability[predicted_class].item()
    return predicted_class, predicted_probability

In [123]:
#정답: 1 / 1 / 1/ 1 / 1 / 0 / 0 / 0 / 0 / 0

s1 = "존잼이다 오컬트 영환데 ㄹㅇ힙함 파묘 보고 김고은과 이도현을 사랑하게 됨ㅠㅠ"
s2 = "배우들 연기 미쳤다.. 몰입하다보니 영화 끝나있음.. 내 기준 올해 한국 영화 중에 탑ㅠ"
s3 = "이런 기이한 이야기에 미술, 의상, 음악이 이렇게 예뻐도 되는 거냐. 스톤과 러팔로도 아카데미 주조연상 후보에 나란히 올라갈 정도의 코미디 연기를 보여줘서 보는 내내 입꼬리가 안 내려갔다"
s4 = "네게 미결로 남고 싶은 내 삶의 흔적"
s5 = "내가 얼마나 '정상적'이고 괜찮은 인간인지 스스로에게, 그리고 타인에게 인정받을 수 있는 가장 쉽고 빠른 방법은 다른 누군가를 괴물이라고 손가락질 해보이는 것이다. 이런식의 괴물 색출, 사냥 놀이에 몰두하는 사회일수록 구성원들의 정신, 사고, 행동은 영화에서처럼 병들고 뒤틀릴 수밖에 없다"
s6 = "명성에 비해 그닥"
s7 = "재밌네요 물론 한번도 웃지는 않았습니다"
s8 = "엥 평론가는 0점 줄수있어요??? 나 cgv vvip 메가박스 mvip 인데 수백편 영화보면서 평론가가 0점 준거 처음봤는데 신기하내요 버그나 오류아니겠죠??평론가 0점 실화에요?? 나는 그래도 솔직히 10점만점에 1.8점 정도인데"
s9 = "맥아리없는 심심한 빌런, 남은건 초롱이뿐"
s10 = "1. 영화 내내 유치하고 재미없는 유머 남발초롱이 캐릭터는 좋았다2. 말이 안 되는 장면 천지초반부터 형사가 일반인 구타?줄빠따 맞고도 아무일 없듯이 칼듯 야쿠자들 관광전편들과 달리 형사로서의 작전 능력 없이 모든 과정이 우연처럼 성공적으로 풀린다3. 약한 빌런임팩트 있게 등장하지만 실상 까보면 아무것도 없음4. 마동석 원맨쇼동료형사들 아예 안 나왔어도 무방할듯광수대로 가면서 동료를 다 버렸다"


In [125]:
reviews = {"review":[s1, s2, s3, s4, s5, s6, s7, s8, s9, s10],
           "answer":[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}
NBoW_score, lstm_score = [], []

for index, review in enumerate(reviews['review']):
    NBoW_pred_class, NBoW_pred_prob = predict_sentiment(NBoW_model, reviews['review'][index])
    lstm_pred_class, lstm_pred_prob = predict_sentiment(lstm_model, reviews['review'][index])
    if NBoW_pred_class==reviews['answer'][index]:

        print("NBoW: "+"%10s"%f"correct: \tpred({NBoW_pred_class}), answer({reviews['answer'][index]})")
        NBoW_score.append(1)
    else:
        print("NBoW: "+"%10s"%f"wrong: \tpred({NBoW_pred_class}), answer({reviews['answer'][index]})")
        NBoW_score.append(0)
    if lstm_pred_class==reviews['answer'][index]:
        print("LSTM: "+"%10s"%f"correct: \tpred({lstm_pred_class}), answer({reviews['answer'][index]})")
        lstm_score.append(1)
    else:
        print("LSTM: "+"%10s"%f"wrong: \tpred({lstm_pred_class}), answer({reviews['answer'][index]})")
        lstm_score.append(0)
print(f"NBoW model: {np.mean(NBoW_score)*100:.0f}점")
print(f"LSTM model: {np.mean(lstm_score)*100:.0f}점")
print('무지성으로 특수문자 제거하니까 성능이 좀 줄어드는 것 같아')

NBoW: correct: 	pred(1), answer(1)
LSTM: correct: 	pred(1), answer(1)
NBoW: correct: 	pred(1), answer(1)
LSTM: correct: 	pred(1), answer(1)
NBoW: wrong: 	pred(0), answer(1)
LSTM: correct: 	pred(1), answer(1)
NBoW: wrong: 	pred(0), answer(1)
LSTM: wrong: 	pred(0), answer(1)
NBoW: correct: 	pred(1), answer(1)
LSTM: correct: 	pred(1), answer(1)
NBoW: correct: 	pred(0), answer(0)
LSTM: correct: 	pred(0), answer(0)
NBoW: wrong: 	pred(1), answer(0)
LSTM: wrong: 	pred(1), answer(0)
NBoW: wrong: 	pred(1), answer(0)
LSTM: wrong: 	pred(1), answer(0)
NBoW: correct: 	pred(0), answer(0)
LSTM: correct: 	pred(0), answer(0)
NBoW: correct: 	pred(0), answer(0)
LSTM: correct: 	pred(0), answer(0)
NBoW model: 60점
LSTM model: 70점
무지성으로 특수문자 제거하니까 성능이 좀 줄어드는 것 같아
